In [32]:
from Scripts.import_data import *
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.svm import SVR
from sklearn.metrics import mean_squared_error
from sklearn.neural_network import MLPRegressor

Zaimportowanie wartości rynkowej

In [ ]:
#market_values to procentowe roczne zmiany wartości spółek, obliczone w notebooku "Market_value_importer"
market_values = pd.read_csv('./Data/market_values.csv')
market_values.set_index('Unnamed: 0', inplace=True)
print(market_values.index)
print(market_values)

Zaimportowanie danych dotyczących alokacji środków

In [20]:
balance_data = import_csv_dict_of_dataframes('./Data/companies_balance_sheet.csv')

for company, years in balance_data.items():
    for year, data in years.items():
        if company in market_values.index and year in market_values.columns:
            # Take only the second level of the MultiIndex
            data.index = data.index.droplevel(0)


Stworzenie jednego zbioru z wszystkimi danymi

In [29]:
columns = ['ASSETS','Non-current assets','Property, plant and equipment','Exploration for and evaluation of mineral resources','Intangible assets','Goodwill','Investment property',
'Right-of-use assets','Investment in affiliates','Non-current financial assets','Non-current loans and receivables','Deferred income tax','Non-current deferred charges and accruals',
'Non-current derivative instruments','Other non-current assets','Current assets','Inventories','Current intangible assets',
'Biological assets','Trade receivables','Loans and other receivables','Financial assets','Cash and cash equivalents','Accruals','Assets from current tax','Derivative instruments',
'Other assets','Assets held for sale and discontinuing operations','Called up capital','Own shares','EQUITY & LIABILITIES','Equity shareholders of the parent','Share capital',
'Called up share capital','Treasury shares','Supplementary capital','Valuation and exchange differences','Other capitals','Retained earnings / accumulated losses','Non-controlling interests',
'Non-current liabilities','Non-current liabilities from derivatives','Non-current loans and borrowings','Non-current liabilities from bonds','Non-current liabilities from finance leases',
'Non-current trade payables','Long-term provision for employee benefits','Deferred tax liabilities','Non-current provision','Other non-current liabilities','Non-current accruals (liability)',
'Current liabilities','Liabilities from derivatives','Financial liabilities (loans and borrowings)','Bond liabilities','Liabilities from finance leases','Trade payables','Employee benefits',
'Current tax liabilities','Provisions','Other liabilities','Accruals (liability)','Liabilities related to assets held for sale and discontinued operations', 'market_value']

rows = []
for company, years in balance_data.items():
    for year, market_data in years.items():
        new_row = {}
        for indicator, value in market_data.items():
            if indicator in columns:
                new_row[indicator] = value
        if company in market_values.index and year in market_values.columns:
            new_row['market_value'] = market_values[year][company]
            rows.append(new_row)

# TODO: zamienić dane o alokacji na procentowe zmiany!!!
data = pd.DataFrame(rows, columns=columns)
print(data)

          ASSETS  Non-current assets Property, plant and equipment  \
0      48649.817           17173.693                       814.332   
1     109262.821  41316.893000000004            18734.064000000002   
2     155668.388  49815.126000000004                     20380.205   
3     186339.222           70397.597                     26889.503   
4     219512.964           90679.674                     25946.479   
...          ...                 ...                           ...   
1240     1778284              994505                        809413   
1241     1790063             1014534                        818103   
1242     2479535             1595330                        905788   
1243     2513606             1729256                       1053811   
1244     2508447             1726721                       1050251   

     Exploration for and evaluation of mineral resources Intangible assets  \
0                                                   NaN          13067.498   
1  

Czyszczenie danych

In [30]:
data = data.fillna(0)
X = data.drop('market_value', axis=1)
y = data['market_value']

Uczenie modelu

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

models = {
    'Linear Regression': LinearRegression(),
    'Random Forest': RandomForestRegressor(),
    'Gradient Boosting': GradientBoostingRegressor(),
    'Support Vector Machine': SVR(kernel='linear'),
    'Neural Network': MLPRegressor(hidden_layer_sizes=(100,), activation='relu', solver='adam', max_iter=1000)
}

# Train and evaluate each model
for name, model in models.items():
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    mse = mean_squared_error(y_test, y_pred)
    print(f'{name} MSE: {mse}')

Linear Regression MSE: 485652.9285892416
Random Forest MSE: 473955.77653803857
Gradient Boosting MSE: 478287.66158511076
